In [ ]:
def measurement_update(posemean, posecov, z):
    
    # 1. Compute measurement Jacobian
    x  = x_check[0,0]
    y  = x_check[0,1]
    th = wraptopi(x_check[0,2])
    
    xl = lk[0]
    yl = lk[1]
    
    star1 =xl - x - d*np.cos(th)
    star2 =yl - y - d*np.sin(th)
    frac = star1**2+star2**2
    den  = np.sqrt(frac)
    
    Hk=np.array([
                [-star1/den,     -star2/den,     (star1*d*np.sin(th)-star2*d*np.cos(th))/den],
                [star2/frac,     -star1/frac,    -1- (d*np.sin(th)*star2/frac) - (d*np.cos(th)*star1/frac)]
                ])
    
    Hk = Hk.reshape((2,3))  
    Mk=np.identity(2)

    # 2. Compute Kalman Gain
    #print('Hk shape is    ' + str(Hk.shape))
    #print('cov_y shape is ' + str(cov_y.shape))
    #print('Mk shape is    ' + str(Mk.shape))
    
    S  = np.matmul(Hk, np.matmul(P_check, Hk.T)) + np.matmul(Mk, np.matmul(cov_y, Mk.T))  
    #S  = np.matmul(Hk, np.matmul(P_check, Hk.T)) + cov_y 
    Kk = np.matmul(np.matmul(P_check, Hk.T), np.linalg.inv(S) )
    #print('Kk shape is    ' + str(Kk.shape))
    
    # 3. Correct predicted state (remember to wrap the angles to [-pi,pi])
    yk=np.array([den, wraptopi(np.arctan2(star2,star1)-wraptopi(th))]).reshape((2,1))  
    ym=np.array([rk,wraptopi(bk)]).reshape((2,1))  
    
    temp = (ym-yk)
    #print('ym shape is    ' + str(ym.shape))
    #print('yk shape is    ' + str(yk.shape))
    #print('temp shape is  ' + str(temp.shape))
    
    #print('residual correction is ' + str(temp))
    x_check = x_check + np.matmul(Kk, temp).T
    #print('x_check shape is    ' + str(x_check.shape))
    x_check[0,2]=wraptopi(x_check[0,2])
        
    # 4. Correct covariance
    P_check= np.matmul( (np.identity(3)-np.matmul(Kk,Hk)) , P_check )
    
    return x_check, P_check
